In [106]:
%pip install nltk


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\ralva\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [133]:
# Lectura de dades (Movie Reviews Corpus) NLTK

import nltk
import re
import sklearn
from sklearn.model_selection import train_test_split 
nltk.download('movie_reviews')
from nltk.corpus import movie_reviews as mr
from nltk.corpus import stopwords
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier


[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ralva\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


In [134]:
all_texts = []
all_categories = []

for fileid in mr.fileids():
    all_texts.append(mr.raw(fileid))
    all_categories.append(mr.categories(fileid)[0])
    


In [135]:
all_categories.count('neg')

1000

In [136]:
# Preprocessing text
def preprocessing(text):
  # Remove digits
  for num in range(10):
    text = text.replace(str(num), "")

  # Pasar tot el text a mínuscules
  text = text.lower()

  # Substituir dos espais per un
  text = re.sub(" +", " ", text)

  # Concatenar frases amb dos espais
  text = text.replace("\n ", "\n  ")

  #Treure tabs
  text = text.replace('\t','')
  
  # Eliminem stopwords al countvectorizer
  
  return text

In [137]:
for text in all_texts:
  text = preprocessing(text)

In [138]:
# Split data into train, test and validation
X_train, X_test, y_train, y_test = train_test_split(all_texts, all_categories, test_size=0.2, random_state=14)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=14)

In [139]:
# Count vectorizer

# Crear un objeto CountVectorizer
vectorizer = CountVectorizer(stop_words=stopwords.words('english'))

# Ajustar el vectorizador a los datos de entrenamiento
train_matrix = vectorizer.fit_transform(X_train)

# Transformar los datos de validación y test
val_matrix = vectorizer.transform(X_val)
test_matrix = vectorizer.transform(y_val)

In [140]:
# Random forest
from sklearn.ensemble import RandomForestClassifier

# Crear un objeto RandomForestClassifier para saber si el texto es positivo o negativo
classifier = RandomForestClassifier(random_state=14)

# Entrenar el clasificador
classifier.fit(train_matrix, y_train)

# Predecir las categorías de los datos de validación
val_predictions = classifier.predict(val_matrix)

In [141]:
for i, prediction in enumerate(val_predictions):
    print(f"Prediction: {prediction}")
    print(f"Real_category: {y_val[i]}")
    print("-----------------------------")
    print(f"Text: {X_val[i]}")

Prediction: pos
Real_category: pos
-----------------------------
Text: once upon a time a solitary ogre named shrek ( mike myers ) was really annoyed to discover that his swamp had been invaded by three blind mice , seven dwarves and all the other fairy tale characters in the land , banished from their home by lord farquaad ( john lithgow ) . 
unwittingly befriended by a non-stop talking donkey ( eddie murphy ) , shrek travels to dulloc to speak with farquaad , where he stumbles into a joust and comes up the winner . 
his prize ? 
to save princess fiona ( cameron diaz ) from a tower surrounded by molten flame and protected by a dragon so she can become farquaad's bride . 
the ogre agrees to the task in return for getting his solitary lifestyle back in " shrek . " 
it took four writers ( ted elliott and terry rossio ( " the road to el dorado " ) , roger s . h . 
schulman ( " balto " ) and joe stillman ( " beavis and butthead do america " ) ) to adapt william steig's children's book . 
t

In [142]:
# Calcular la precisión
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_val, val_predictions)

accuracy

0.78125

In [143]:
def predict_sentiment(text):
  text = preprocessing(text)
  text_matrix = vectorizer.transform([text])
  prediction = classifier.predict(text_matrix)
  return prediction[0]

In [144]:
predict_sentiment("""The word that will likely be used most often to describe Denis Villeneuve’s “Dune: Part Two” is “massive.” Expect a whole lot of variations on the words “epic” and “spectacle” too. Whatever big words you apply to the result, Villeneuve undeniably did not approach Frank Herbert’s beloved sci-fi novel with modest aspirations, and it’s his ambition, along with the top tier of behind-the-scenes craftspeople with whom he collaborated, that have paid off in this superior follow-up to the Oscar-winning 2021 film. While that beloved blockbuster often felt like half a film, “Dune: Part Two” locates significantly higher stakes on Arrakis, while injecting just enough humor and nuanced themes about power and fanaticism to flavor the old-fashioned storytelling. More than a simple savior or chosen one story, “Dune: Part Two” is a robust piece of filmmaking, a reminder that this kind of broad-scale blockbuster can be done with artistry and flair.

“Dune: Part Two” picks up so closely on the heels of the first film that the Fremen are still transporting the body of Jamis (Babs Olusanmokun) home again after he was bested in the fight with Paul Atreides (Timothee Chalamet). After the massacre of House Atreides, Paul chose to go with the Fremen, much to the consternation of his mother Jessica (Rebecca Ferguson). Thinking both Paul and Jessica were taken by the desert and all hopped up on violence after destroying the Atreides interlopers, House Harkonnen amplifies its attack on the Fremen, leading to a few remarkably staged battles between the warriors and soldiers. Villeneuve and his team deftly fill the first hour with battle sequences that counter the firepower of the Harkonnen military and the Fremen tribal combatants, who often literally emerge from the earth to destroy them. Bodies fall from the sky as enormous ships burst into flames in a way that feels nearly operatic. Amidst the chaos, Dave Bautista cannily sketches Rabban Harkonnen as a wartime leader who is in way over his bald head while Stellan Skargard leans even harder into a sort of blend between Nosferatu and Jabba the Hutt.

As the battle between the Fremen and the Harkonnens for control of Arrakis serves as the backdrop for “Dune: Part Two,” Paul’s arc from nervous young man at the beginning of the first film to potential leader plays out in the foreground. A Fremen tribal leader named Stilgar (Javier Bardem) is convinced that Paul Atreides is the chosen one that has been foretold among his people for generations. Even as so much of the mythology points to Paul’s savior role, the Emo King tries to blend into the Fremen, forming a relationship with a young warrior named Chani (Zendaya). Paul passes the tests put in front of him by the Fremen, takes on the tribal name of Muad’Dib, and vows vengeance against the Harkonnens who were behind his father’s death.

On another planet, an Emperor named Shaddam IV (Christopher Walken) counsels with his daughter Irulan (Florence Pugh) and a Bene Gesserit Reverend Mother (Charlotte Rampling) on the state of Arrakis. It’s revealed early on that Shaddam basically sent House Atreides to its destruction, meaning he’s on that vengeance list that Paul’s been keeping, while Irulan serves as a sort-of narrator for “Dune: Part Two,” dictating some of the political developments into a device that’s really designed to keep audiences with the plot.

If the interstellar politics aren’t enough, writers Villeneuve and Jon Spaihts inject a nice dose of religious fanaticism for the inevitable think pieces too. Lady Jessica becomes a powerful religious figure of her own among the Fremen, guiding her son’s ascendance in a manner that feels nefarious and unsettling. “Dune: Part Two” is not a traditional hero’s journey in that it’s constantly questioning if being led by an outsider from another culture is the right move—Chani sure doesn’t think so, and Zendaya subtly finds notes to make viewers wonder what a happy ending would be for these characters. As Jessica and Paul learn more about Fremen history and culture, they threaten not to lead it as much as dismantle and own it. There’s a big difference.

While the plotting in “Part Two” is undeniably richer than the first film, its greatest assets are once again on a craft level. Greig Fraser, who won the Oscar for cinematography the first time, tops his work there with stunning use of color and light. It’s in the manner the sun hits Chalamet’s face at a certain angle or the wildly different palettes that differentiate the Harkonnens and the Fremen. The browns and blues of the desert culture don’t feel arid as much as grounded and tactile, while the Harkonnen world is so devoid of color that it’s often literally black and white—even what look like fireworks pop like someone throwing colorless paint at a wall. Hans Zimmer’s Oscar-winning score felt a bit overdone to me in the first film, but he smartly differentiates the cultures here, finding more metallic sounds for the cold Harkonnens to balance against the heated score for the Fremen. Finally, the effects and sound design feel denser this time, and the fight choreography reminds one how poorly this has been done in other blockbuster films.

As for performers, Chalamet is likely to be the most divisive element, often feeling a bit flat for someone believed to be the Neo of this world. However, those choices add up in a way that makes thematic sense, enhancing the uncertainty of Paul’s rise. Zendaya is solid—although she lacks chemistry with Chalamet that would have helped—but it’s Ferguson’s slippery performance and Bardem’s playful one that really add flavors here that weren’t in the first outing. Finally, Austin Butler leans hard into the exaggerated role of Feyd-Rautha, playing the sociopathic nephew of the Baron with all the scenery-chewing intensity that a character like this needs to work, finding the emotional void to balance out against Chalamet’s tempestuous inner monologue.

“Dune: Part Two” has been compared to “The Empire Strikes Back” in the run-up to its release, and that’s not quite right. The better comparison is “The Lord of the Rings: The Two Towers,” another film that built on what we knew about the characters from the first film, added a few new ones, and really amplified a sense of continuous battle and danger. Like both films, a third chapter feels inevitable. Critics will have to come up with a new synonym for massive.""")

'pos'

In [145]:
# EBM classifier
EBM = ExplainableBoostingClassifier(random_state=14)

EBM.fit(train_matrix, y_train)

val_predictions_EBM = EBM.predict(val_matrix)

accuracy_EBM = accuracy_score(y_val, val_predictions_EBM)